In [2]:
# !pip install fake_useragent

### Selenium으로 크롤링

In [30]:
# !pip install selenium

  Using cached https://files.pythonhosted.org/packages/80/d6/4294f0b4bce4de0abf13e17190289f9d0613b0a44e5dd6a7f5ca98459853/selenium-3.141.0-py2.py3-none-any.whl


In [104]:
from selenium import webdriver

In [105]:
url= 'http://www.zdnet.co.kr/newskey/?lstcode=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'

In [59]:
driver =  webdriver.Chrome()
driver.get(url)

In [122]:
#여러데이터 모으기
articles = driver.find_elements_by_css_selector("body > div.contentWrapper > div > div.left_cont > section > div")
len(articles)

17

In [123]:
#제목만 뽑아내기
import pandas as pd

title_list = []
for article in articles:
    title = article.find_element_by_css_selector("div.assetText > a > h3").text 
    title_list.append(title)
title =pd.DataFrame(title_list, columns=['title'])
title.head()

,title
0,"""AI, 일자리를 빼앗지 않고 일하는 방식 혁신할 것"""
1,AI 시대 이용자보호 정책 전담조직 생긴다
2,"산업부, 디지털통상 전문인력 양성한다"
3,"구글 딥마인드 ""의사보다 유방암 오판율 낮은 AI모델 개발"""
4,마이너리티 리포트가 현실로…'AI+CCTV'로 범죄 예측 가능해진다


In [124]:
#링크만 뽑아내기

link_list = []
for article in articles:
    link = article.find_element_by_css_selector("a").get_attribute("href")
    link_list.append(link)
link= pd.DataFrame(link_list, columns=['link'])
link.head() #상위 5개 보여줌

,link
0,http://www.zdnet.co.kr/view/?no=20200102141508
1,http://www.zdnet.co.kr/view/?no=20200102111519
2,http://www.zdnet.co.kr/view/?no=20200102110525
3,http://www.zdnet.co.kr/view/?no=20200102101522
4,http://www.zdnet.co.kr/view/?no=20200102091441


In [125]:
#이미지만 뽑아내기

img_list = []
for article in articles:
    img = article.find_element_by_css_selector("div.assetThumb > a > figure > img").get_attribute("src") 
    img_list.append(img)
img=pd.DataFrame(img_list,columns=['img'])
img.head()

,img
0,http://image.zdnet.co.kr/2020/01/02/sangheek_e...
1,http://image.zdnet.co.kr/2020/01/02/psooh_XzAy...
2,http://image.zdnet.co.kr/2019/11/11/mjjoo_oNiO...
3,http://image.zdnet.co.kr/2018/02/23/sangheek_c...
4,http://image.zdnet.co.kr/2020/01/02/jiny_WCHYi...


In [126]:
result=pd.concat([img, title, link], axis=1)
result.tail() #하위5개 보여줌

,img,title,link
12,http://image.zdnet.co.kr/2019/12/18/insight_Jm...,"'반도체 굴기' 성과 낸 中..""신기술 확보 필요해""",http://www.zdnet.co.kr/view/?no=20191231105746
13,http://image.zdnet.co.kr/2019/12/31/sun1108_wp...,최기영 장관 “2020년 AI 강국·미디어 생태계 조성”,http://www.zdnet.co.kr/view/?no=20191231103105
14,http://image.zdnet.co.kr/2019/12/31/hjyoo_8saf...,中 베이징서 '기사없는 택시' 등장한다,http://www.zdnet.co.kr/view/?no=20191231090244
15,http://www.zdnet.co.kr/images/default_img.jpg,"NIA, '공공기관 신뢰가능 AI 구현 실용가이드' 발간",http://www.zdnet.co.kr/view/?no=20191230152910
16,http://image.zdnet.co.kr/2019/12/10/imc_I9Uhii...,"SAS, 2020년 데이터 분석 대중화 목표",http://www.zdnet.co.kr/view/?no=20191230133412


In [119]:
# 함수로 만들기
def get_articles(page):
    driver =  webdriver.Chrome()
    url = 'http://www.zdnet.co.kr/newskey/?lstcode=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&page={}'.format(page)
    driver.get(url)
    #여러데이터 모으기
    articles = driver.find_elements_by_css_selector("body > div.contentWrapper > div > div.left_cont > section > div")
    
    #제목
    title_list = []
    for article in articles:
        title = article.find_element_by_css_selector("div.assetText > a > h3").text 
        title_list.append(title)
    title =pd.DataFrame(title_list, columns=['title'])
    
    #링크만 뽑아내기
    link_list = []
    for article in articles:
        link = article.find_element_by_css_selector("a").get_attribute("href")
        link_list.append(link)
    link= pd.DataFrame(link_list, columns=['link'])
    
    #이미지만 뽑아내기
    img_list = []
    for article in articles:
        img = article.find_element_by_css_selector("div.assetThumb > a > figure > img").get_attribute("src") 
        img_list.append(img)
    img=pd.DataFrame(img_list,columns=['img'])
    
    result=pd.concat([img, title, link], axis=1)
    driver.quit()
    return result

In [142]:
# 자동으로 일정 페이지 크롤링해줘서 데이터프레임 만들어주기
def zd_net(startpage, endpage):
    df=pd.DataFrame([])
    for i in range(startpage, endpage+1):
        temp=get_articles(i)
        df= pd.concat([df,temp],axis=0)
    #index 번호 다시 매기기
    df.reset_index(drop=True, inplace=True)
    return df
      

In [143]:
final_df= zd_net(1,4)

In [144]:
final_df.to_csv("zd_net.csv", index=False, encoding='utf-8' )

In [146]:
## 참고
### csv파일을 여기서 읽고 싶다면
final_df = pd.read_csv("zd_net.csv", encoding="utf-8")


### 만약 엑셀에서 열고 싶다면
1. 엑셀 열기
2. 데이터 탭 > 외부데이터 가져오기> 아무거나 누르기> 모든파일> csv 파일선택
3. 언어를 utf-8로 설정하고 열기